In [3]:
dadagp_path = '/Users/pedro/Desktop/DadaGP-v1.1/'
all_files_path = '/Users/pedro/Desktop/DadaGP-v1.1/_DadaGP_all_filenames.json'

## Get all .gp files

In [12]:
import json

# Assuming 'all_files_path' contains the path to your JSON file
with open(all_files_path, 'r') as f:
    file_paths = json.load(f)  # Load JSON data into a Python list

# Remove the ".tokens.txt" part from each path
all_gps_paths = [path.replace(".tokens.txt", "") for path in file_paths]

# Print the number of files
print(len(all_gps_paths))

26181


## Get all .gp files with => 4 tracks

In [25]:
import guitarpro as gp
import os
from tqdm import tqdm

gp_files_4plus = []

# Iterate over the list of .gp file paths
for gp_file_path in tqdm(all_gps_paths):
    # Get full path
    file_path = os.path.join(dadagp_path, gp_file_path)
    # Parse them with pyguitarpro
    try:
        # Parse the Guitar Pro file
        gp_file = gp.parse(file_path)

        # Check the number of tracks
        if len(gp_file.tracks) >= 4:
            gp_files_4plus.append(file_path)
            # print('File {} has {} tracks.'.format(file_path, len(gp_file.tracks)))

    except Exception as e:
        # Handle any parsing errors gracefully
        print('Could not parse {}: {}'.format(file_path, e))
    

100%|██████████| 26181/26181 [30:38<00:00, 14.24it/s]  


In [33]:
print('There are {} files with 4 or more tracks, which is {:.2f}% of the dataset.'.format(len(gp_files_4plus), 100*len(gp_files_4plus)/len(all_gps_paths)))

There are 13413 files with 4 or more tracks, which is 51.23% of the dataset.


In [34]:
# Save gp_files_4plus into a .json
with open("gp_paths_4plustracks.json", "w") as final:
	json.dump(gp_files_4plus, final)

## Remove extra tracks from .gp files

In [38]:
# If a track only has 2 distorted guitars, bass, drums -> save the track
# If a track has more than those instruments, check if it has 2 DGs, B, D -> save those in new file

with open('gp_paths_4plustracks.json', 'r') as f:
    gp_files_4plus = json.load(f)  # Load JSON data into a Python list

13413


In [115]:
gp_files_4plus[666]

'/Users/pedro/Desktop/DadaGP-v1.1/A/Angra/Angra - Speed (3).gp3'

In [123]:
gp_file = gp.parse(gp_files_4plus[666])
print(gp_file)

gp_file

AttributeError: 'Song' object has no attribute 'Song'

In [122]:
required_tracks 

for t in required_tracks:
    a = gp.write(t, 'a.gp5')

a

AttributeError: 'Track' object has no attribute 'clipboard'

In [119]:
import os
import shutil

# Directory to save filtered files
output_directory = "/Users/pedro/Desktop/DadaGP-v1.1/data-prep-acc-gen/gps_with_4"

# Make sure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# MIDI Channel numbers for the required instruments
DISTORTED_GUITAR_CHANNELS = [30, 31]  # Distorted/Overdriven guitars
BASS_CHANNEL = [33,34]  # Finger/Picked Bass
# No need to define drums channel as we'll use isPercussionTrack for this

# Function to check if a track is a distorted guitar, bass, or drums
def is_required_instrument(track):
    instrument = track.channel.instrument
    if track.isPercussionTrack:  # Check if it's a percussion (drum) track
        return 'drums'
    elif instrument in DISTORTED_GUITAR_CHANNELS:
        return 'distorted_guitar'
    elif instrument in BASS_CHANNEL:
        return 'bass'
    
    return None


try:
    # Parse the Guitar Pro file
    # print(gp_files_4plus[666])
    gp_file = gp.parse(gp_files_4plus[666])
    print(gp_file)
    gp_filename = gp_files_4plus[666]
    print(gp_filename)

    # Initialize lists to store required instruments
    distorted_guitars = []
    bass = None
    drums = None

    # Filter tracks based on MIDI channel and percussion check
    for track in gp_file.tracks:
        # print(track)
        instrument_type = is_required_instrument(track)
        if instrument_type == 'distorted_guitar':
            distorted_guitars.append(track)
        elif instrument_type == 'bass' and bass is None:
            bass = track
        elif instrument_type == 'drums' and drums is None:
            drums = track

    print(distorted_guitars)
    print(bass)
    print(drums)

    # Check if we have the required tracks (2 distorted guitars, 1 bass, 1 drums)
    if len(distorted_guitars) == 2 and bass and drums:
        required_tracks = distorted_guitars + [bass, drums]
        print(required_tracks)

        if len(gp_file.tracks) == 4:
            print('HERE!')
            # If the file already has exactly 4 tracks, just copy it
            output_file = os.path.join(output_directory, os.path.basename(file_path))
            shutil.copy(file_path, output_file)
            print("Copied {} to {}.".format(file_path, output_file))
        else:
            print('HERE-NOT')
            # If there are more than 4 tracks, create a new file with the filtered tracks
            new_gp_file_tracks = required_tracks  # Set the filtered tracks
            new_gp_file = gp.write(new_gp_file_tracks, 'a.gp5')

            # Save the new file with '_4tracks' appended
            # file_name, file_extension = os.path.splitext(os.path.basename(file_path))
            # new_file_path = os.path.join(output_directory, "{}_4tracks{}".format(file_name, file_extension))
            # gp.write(new_gp_file, new_file_path)

            print("Created new file with 4 tracks: {}".format(new_file_path))

except Exception as e:
    print("Could not process {}: {}".format(file_path, e))



/Users/pedro/Desktop/DadaGP-v1.1/A/Angra/Angra - Speed (3).gp3
[<guitarpro.models.Track object at 0x7fc6c1838f60>, <guitarpro.models.Track object at 0x7fc6c183a240>]
[<guitarpro.models.Track object at 0x7fc6c1838f60>, <guitarpro.models.Track object at 0x7fc6c183a240>, <guitarpro.models.Track object at 0x7fc6c183a4e0>, <guitarpro.models.Track object at 0x7fc6c183af60>]
HERE-NOT
Could not process /Users/pedro/Desktop/DadaGP-v1.1/Z/Zyklon/Zyklon - Psyklon Aeon.gp4: 'list' object has no attribute 'Song'


In [86]:
print(gp_file.tracks[0].isPercussionTrack)
print(gp_file.tracks[2].channel.instrument)

False
33


In [91]:
gp_file.tracks[2].channel

def is_required_instrument(track):
    instrument = track.channel.instrument
    if track.isPercussionTrack:  # Check if it's a percussion (drum) track
        return 'drums'
    elif instrument in DISTORTED_GUITAR_CHANNELS:
        return 'distorted_guitar'
    elif instrument in BASS_CHANNEL:
        return 'bass'
    
    return None

is_required_instrument(gp_file.tracks[6])


'drums'